In [ ]:
import React from 'react';
import { motion } from 'framer-motion';
import { format } from 'date-fns';
import { Clock, Flame, ChevronRight, Utensils } from 'lucide-react';
import { Badge } from "@/components/ui/badge";

const mealTypeConfig = {
  breakfast: { label: 'Breakfast', icon: '🌅', color: 'bg-amber-100 text-amber-700' },
  lunch: { label: 'Lunch', icon: '☀️', color: 'bg-emerald-100 text-emerald-700' },
  dinner: { label: 'Dinner', icon: '🌙', color: 'bg-indigo-100 text-indigo-700' },
  snack: { label: 'Snack', icon: '🍎', color: 'bg-pink-100 text-pink-700' },
};

const MealCard = ({ scan, onClick }) => {
  const config = mealTypeConfig[scan.meal_type] || mealTypeConfig.snack;
  
  return (
    <motion.div 
      initial={{ opacity: 0, x: -20 }}
      animate={{ opacity: 1, x: 0 }}
      whileHover={{ scale: 1.02 }}
      onClick={onClick}
      className="bg-white rounded-2xl p-4 border border-slate-100 shadow-sm cursor-pointer flex items-center gap-4 group"
    >
      {scan.image_url ? (
        <img 
          src={scan.image_url} 
          alt={scan.food_name}
          className="w-16 h-16 rounded-xl object-cover flex-shrink-0"
        />
      ) : (
        <div className="w-16 h-16 rounded-xl bg-slate-100 flex items-center justify-center flex-shrink-0">
          <Utensils className="w-6 h-6 text-slate-400" />
        </div>
      )}
      
      <div className="flex-1 min-w-0">
        <div className="flex items-center gap-2 mb-1">
          <Badge className={`${config.color} border-0 text-xs`}>
            {config.icon} {config.label}
          </Badge>
        </div>
        <h4 className="font-semibold text-slate-800 truncate">{scan.food_name}</h4>
        <div className="flex items-center gap-3 mt-1 text-sm text-slate-500">
          <span className="flex items-center gap-1">
            <Flame className="w-3.5 h-3.5 text-orange-400" />
            {scan.calories} kcal
          </span>
          <span className="flex items-center gap-1">
            <Clock className="w-3.5 h-3.5" />
            {format(new Date(scan.created_date), 'h:mm a')}
          </span>
        </div>
      </div>
      
      <ChevronRight className="w-5 h-5 text-slate-300 group-hover:text-slate-400 transition-colors" />
    </motion.div>
  );
};

export default function MealHistory({ scans = [], onSelectMeal }) {
  const groupedByDate = (scans || []).reduce((acc, scan) => {
    const date = scan.scan_date || format(new Date(scan.created_date), 'yyyy-MM-dd');
    if (!acc[date]) acc[date] = [];
    acc[date].push(scan);
    return acc;
  }, {});

  const sortedDates = Object.keys(groupedByDate).sort((a, b) => new Date(b) - new Date(a));

  if (!scans || scans.length === 0) {
    return (
      <div className="bg-slate-50 rounded-3xl p-8 text-center">
        <div className="w-16 h-16 bg-slate-100 rounded-2xl flex items-center justify-center mx-auto mb-4">
          <Utensils className="w-8 h-8 text-slate-400" />
        </div>
        <h3 className="text-lg font-semibold text-slate-800 mb-2">No meals logged yet</h3>
        <p className="text-slate-500 text-sm">Scan your first meal to start tracking</p>
      </div>
    );
  }

  return (
    <div className="space-y-6">
      {sortedDates.map(date => {
        const isToday = date === format(new Date(), 'yyyy-MM-dd');
        const dateLabel = isToday ? 'Today' : format(new Date(date), 'EEEE, MMM d');
        const dayScans = groupedByDate[date];
        const totalCalories = dayScans.reduce((sum, s) => sum + (s.calories || 0), 0);

        return (
          <div key={date}>
            <div className="flex items-center justify-between mb-3">
              <h3 className="text-sm font-medium text-slate-500">{dateLabel}</h3>
              <span className="text-sm text-slate-400">{totalCalories} kcal total</span>
            </div>
            <div className="space-y-3">
              {dayScans.map(scan => (
                <MealCard 
                  key={scan.id} 
                  scan={scan} 
                  onClick={() => onSelectMeal?.(scan)}
                />
              ))}
            </div>
          </div>
        );
      })}
    </div>
  );
}